In [2]:
from dashboard.logic.io import GSHEETS_URL, read_gsheet, comment_button
import pandas as pd
import numpy as np

df = read_gsheet(
    GSHEETS_URL, 
    header=None
)

In [3]:
def findRowColRegex(df: pd.DataFrame, pat: str, case: bool=True, regex:bool=True):
    
    # make sure only one such pat exists in the df
    df_mask = df.apply(lambda x: x.str.contains(pat, case=case, regex=regex) if x.dtype == 'object' else None)
    
    count = df_mask.sum().sum()
    
    if count < 1: 
        raise ValueError(f'Given {pat} did not give any results.')
    if count > 1:
        raise ValueError(f'Given {pat} gave more than 1 results.')
     
    # col and row values where pat
    row = df_mask.any(axis='columns').argmax()
    col = df_mask.any(axis='index').argmax()
    
    return row, col

def getDataFrames(df: pd.DataFrame):

    df = df.reset_index(drop=True)
    dataframes_dict = {}
    
    # MAIN DF
    main_df_idx2 = (df == 'Monthly Income').any(axis='columns').argmax()
    dataframes_dict['main'] = df.iloc[:main_df_idx2+1,:3].dropna(how='all', axis='rows')
    
    # ADS DF
    r1, c1 = findRowColRegex(df, "My Finance Course")
    
    
    ads_df = df.iloc[r1:,c1:].reset_index(drop=True)
    ads_df = (ads_df
        .loc[:(ads_df[c1].isna()).argmax()-1, :]
        .dropna(how='all', axis='columns')
    )
    dataframes_dict['ads'] = ads_df
    
    # OTHER DFS
    df_names = [
    'announcements',
    'advice',
    'error_warning',
    'risk',
    'skip',
    'skip',
    'skip',
    'historical',
    'cash_pos',
    'general_notes',
    'success',
    ]
    
    # Find rows after main df where all values are NaN-s.
    nan_rows = df.iloc[main_df_idx2+1:,].isna().all(axis='columns')
    
    # Indices where all NaN-s in the row
    dfs_idxs = []
    
    for i,bool in nan_rows.items():
        
        if i == nan_rows.index[-1]:
            dfs_idxs.append(None)
            break
        if bool:
            dfs_idxs.append(i)
    
    # raw dataframes
    dfs = []        
    for i,idx in enumerate(dfs_idxs):
        if i == len(dfs_idxs) - 1:
            break
        dfs.append(df.iloc[idx:dfs_idxs[i+1]])
    
    # strip dataframes from NaN-s and add to dictionary
    for df_, name in zip(dfs, df_names):
        
        if name == 'skip': continue
        
        dataframes_dict[name] = (df_
            .dropna(how='all', axis='columns')
            .dropna(how='all', axis='rows')
        )
    
    return dataframes_dict

df1 = df.copy()

#findRowColRegex(df1, 'My Finance Course')

df_dict = getDataFrames(df1)
df_dict.keys()

dict_keys(['main', 'ads', 'announcements', 'advice', 'error_warning', 'risk', 'historical', 'cash_pos', 'general_notes', 'success'])

In [185]:
df_risk = df_dict['risk'].copy()
df_risk = df_risk.iloc[1:-5]
df_risk.columns = ['asset', 'risk', 'comment']
df_risk.iloc[-2].comment += ". " + df_risk.iloc[-1, -1]
df_risk = df_risk.iloc[:-1].reset_index(drop=True)
df_risk['risk'] = [1, 1, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4]
df_risk['risk_word'] = df_risk['risk'].replace({1:'VERY LOW', 2: 'LOW', 3: 'MEDIUM', 4: 'HIGH'})
df_risk['li_group'] = df_risk['risk'].replace({1: 'success', 2: 'warning', 3: 'info', 4: 'danger'})
df_risk







,asset,risk,comment,risk_word,li_group
0,Silver,1,Unlikely to be confiscated & good inflation he...,VERY LOW,success
1,Farmland,1,Holds value extremely well and can be used for...,VERY LOW,success
2,Gold,2,Risk of Confiscation under a GOLD standard sce...,LOW,warning
3,Crypto,3,Risk of further/continued regulation or the cu...,MEDIUM,info
4,Cash,3,WILL be phased out when CBDC launches (3-5 yea...,MEDIUM,info
5,Bonds,3,Russia is a great example of paper certificate...,MEDIUM,info
6,Businesses,3,Inflation = Consumer cut in spending = RECESSI...,MEDIUM,info
7,Pensions,3,Collapse risk! (Watch my YT video on this here...,MEDIUM,info
8,Real Estate,4,High risk of declines in 2023 - but each Count...,HIGH,danger
9,Stock Markets,4,High risk of declines once USA interest rates ...,HIGH,danger


In [184]:
for i in df_risk.iloc[0]:
    print(i)

Silver
VERY LOW
Unlikely to be confiscated & good inflation hedge (but be careful with investing in silver mining stocks as they could crash hard before recovering)
